<a href="https://colab.research.google.com/github/BinhLe001/FantasyPL/blob/main/DraftAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=09f10d72f4704a3afa37a467feb4fa865f49dade85667e19b94c7a8d39fc41e6
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 19 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package

In [12]:
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.types as pyspark_types
import pyspark.sql.functions  as psf
from pyspark import SparkContext, SparkConf

In [96]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050").set("spark.driver.memory", "5g")

# uncomment to recreate the context
# spark.stop()
# sc.stop()
# create the context
sc = SparkContext.getOrCreate(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [6]:
import json
import requests

In [7]:
fpl_league_details = json.loads(requests.get('https://draft.premierleague.com/api/league/46331/details').text)
fpl_league_details.keys()

dict_keys(['league', 'league_entries', 'matches', 'standings'])

In [97]:
league_entries = spark.createDataFrame(fpl_league_details["league_entries"]).withColumnRenamed("id", "league_entry").drop("joined_time")
league_entries.show()

+--------+-------------------+------------+-----------------+----------------+----------+-----------+
|entry_id|         entry_name|league_entry|player_first_name|player_last_name|short_name|waiver_pick|
+--------+-------------------+------------+-----------------+----------------+----------+-----------+
|  172408|NamsArmyOfChampions|      173108|              Nam|              Le|        NL|         10|
|  172479|       Weebs United|      173181|             Binh|              Le|        BL|          5|
|  178032|         Relegation|      178756|             Nick|          Wester|        NW|          7|
|  178525|     Stinkcopper FC|      179251|           taylor|         kimbrel|        tk|          2|
|  218922| I <3 White Rice FC|      219906|               An|              Le|        AL|          8|
|  245921|        Chadvera FC|      247072|             Chad|        Verastek|        CV|          3|
|  273160|  Winnie Wunderkind|      274589|             Eric|         Nartker|    

In [98]:
standings = spark.createDataFrame(fpl_league_details["standings"]).join(league_entries.select("league_entry", "entry_name"), how="left", on="league_entry")
standings.show()

+------------+---------+-------------+------------+--------------+-----------+--------------+----------+----+---------+-----+-------------------+
|league_entry|last_rank|matches_drawn|matches_lost|matches_played|matches_won|points_against|points_for|rank|rank_sort|total|         entry_name|
+------------+---------+-------------+------------+--------------+-----------+--------------+----------+----+---------+-----+-------------------+
|      276578|        5|            0|           5|            38|          5|           387|       387|   5|        5|   15|             CRG FC|
|      274589|        1|            0|           3|            38|          7|           392|       457|   2|        2|   21|  Winnie Wunderkind|
|      178756|        4|            0|           4|            38|          6|           402|       411|   4|        4|   18|         Relegation|
|      219906|        3|            0|           3|            38|          7|           366|       441|   3|        3|   21

In [99]:
def create_non_null_dataset(json_data):
  non_null_keys = set()
  for event in json_data:
    for key, value in event.items():
        if value:
          non_null_keys.add(key)
  non_null_data = []
  for event in json_data:
    new_event = {}
    for key, value in event.items():
      if key in non_null_keys:
        new_event[key] = value
    non_null_data.append(new_event)
  return non_null_data

In [101]:
matches_json = create_non_null_dataset(fpl_league_details["matches"])
matches = spark.createDataFrame(matches_json)
matches_league_entries = league_entries.select("league_entry", "entry_name")
matches = (matches.join(matches_league_entries, how="left", on=matches_league_entries.league_entry == matches.league_entry_1)
                  .withColumnRenamed("entry_name", "entry_name_1").drop("league_entry")
                  .join(matches_league_entries, how="left", on=matches_league_entries.league_entry == matches.league_entry_2)
                  .withColumnRenamed("entry_name", "entry_name_2").drop("league_entry")
                  .orderBy(psf.asc("event"))
)
matches.show()

+-----+--------+--------------+---------------------+--------------+---------------------+-------+-------------------+-------------------+
|event|finished|league_entry_1|league_entry_1_points|league_entry_2|league_entry_2_points|started|       entry_name_1|       entry_name_2|
+-----+--------+--------------+---------------------+--------------+---------------------+-------+-------------------+-------------------+
|    1|    true|        276210|                   56|        247072|                   42|   true|  The Pecan Sandies|        Chadvera FC|
|    1|    true|        173181|                   26|        178756|                   37|   true|       Weebs United|         Relegation|
|    1|    true|        276578|                   20|        219906|                   38|   true|             CRG FC| I <3 White Rice FC|
|    1|    true|        274589|                   43|        173108|                   62|   true|  Winnie Wunderkind|NamsArmyOfChampions|
|    1|    true|        293

In [106]:
choices = json.loads(requests.get('https://draft.premierleague.com/api/draft/46331/choices').text)
static = json.loads(requests.get('https://draft.premierleague.com/api/bootstrap-static').text)

In [128]:
draft = spark.createDataFrame(create_non_null_dataset(choices["choices"])).withColumnRenamed("entry_name", "team_name").withColumnRenamed("element", "element_id")
draft.show()

+--------------------+----------+------+-------------------+-------+-----+------+----+-----------------+----------------+-----+--------+
|         choice_time|element_id| entry|          team_name|     id|index|league|pick|player_first_name|player_last_name|round|was_auto|
+--------------------+----------+------+-------------------+-------+-----+------+----+-----------------+----------------+-----+--------+
|2023-08-07T01:00:...|       355|172408|NamsArmyOfChampions|2930071|    1| 46331|   1|              Nam|              Le|    1|   false|
|2023-08-07T01:01:...|       308|275105|             CRG FC|2930072|    2| 46331|   2|        Christian|           Farag|    1|   false|
|2023-08-07T01:01:...|       349|292200|          EnzoMatic|2930073|    3| 46331|   3|            Scott|         Solomon|    1|   false|
|2023-08-07T01:01:...|       500|178032|         Relegation|2930074|    4| 46331|   4|             Nick|          Wester|    1|   false|
|2023-08-07T01:02:...|       396|178525| 

In [120]:
positions = spark.createDataFrame(static["element_types"]).select(psf.col("id").alias("element_type"), psf.col("plural_name").alias("position"))
positions.show()

+------------+-----------+
|element_type|   position|
+------------+-----------+
|           1|Goalkeepers|
|           2|  Defenders|
|           3|Midfielders|
|           4|   Forwards|
+------------+-----------+



In [131]:
elements = spark.createDataFrame(create_non_null_dataset(static["elements"])).join(positions, on="element_type", how="left").withColumnRenamed("id", "element_id")
elements.show()

+------------+--------------------+-------+-----+---+----------------------------+----------------------------+------------+------+----------+---------------+--------------------+----------+------------+----------------+--------------------------+--------------+-----------------------+----------+----+--------------+------------+---------+--------------+-------------------+----------+------------+---------+--------------+-------------------+-------+--------------------+--------------------+---------+----------------+---------------+---------------+---------+-----+--------------------+------+------+----+------+-----------+----------------+------------+------------+------------+-----------+
|element_type|               added|assists|bonus|bps|chance_of_playing_next_round|chance_of_playing_this_round|clean_sheets|  code|creativity|creativity_rank|creativity_rank_type|draft_rank|event_points|expected_assists|expected_goal_involvements|expected_goals|expected_goals_conceded|first_name|form|g

In [134]:
# TOTAL POINTS PER TEAM BASED ON DRAFT
draft_elements = draft.join(elements, how="left", on="element_id")
draft_elements.groupBy("team_name").agg(psf.sum("total_points").alias("sum_points")).orderBy(psf.desc("sum_points")).show()

+-------------------+----------+
|          team_name|sum_points|
+-------------------+----------+
|NamsArmyOfChampions|       484|
|       Weebs United|       481|
|  The Pecan Sandies|       465|
|  Winnie Wunderkind|       441|
| I <3 White Rice FC|       441|
|        Chadvera FC|       384|
|     Stinkcopper FC|       363|
|             CRG FC|       357|
|          EnzoMatic|       336|
|         Relegation|       284|
+-------------------+----------+



In [136]:
# TOTAL POINTS PER TEAM PER POSITION BASED ON DRAFT
position_names = positions.select("position").rdd.flatMap(lambda x: x).collect()
for position in position_names:
  print("POSITION: " + position)
  draft_elements.filter(psf.col("position") == position).groupBy("team_name").agg(psf.sum("total_points").alias("sum_points")).orderBy(psf.desc("sum_points")).show()

POSITION: Goalkeepers
+-------------------+----------+
|          team_name|sum_points|
+-------------------+----------+
|        Chadvera FC|        76|
|  The Pecan Sandies|        69|
|  Winnie Wunderkind|        69|
|       Weebs United|        64|
| I <3 White Rice FC|        64|
|     Stinkcopper FC|        50|
|NamsArmyOfChampions|        41|
|          EnzoMatic|        38|
|             CRG FC|        27|
|         Relegation|         0|
+-------------------+----------+

POSITION: Defenders
+-------------------+----------+
|          team_name|sum_points|
+-------------------+----------+
|NamsArmyOfChampions|       158|
|          EnzoMatic|       149|
|       Weebs United|       142|
| I <3 White Rice FC|       131|
|             CRG FC|       129|
|  The Pecan Sandies|       112|
|         Relegation|       104|
|  Winnie Wunderkind|        98|
|        Chadvera FC|        75|
|     Stinkcopper FC|        68|
+-------------------+----------+

POSITION: Midfielders
+---------